# Importar Librerías Necesarias
Importar las librerías necesarias como pandas, numpy, matplotlib, seaborn, sklearn, y nltk.

In [2]:
import pandas as pd  # Librería para manipulación de datos
import numpy as np  # Librería para operaciones numéricas
import matplotlib.pyplot as plt  # Librería para visualización de datos
import seaborn as sns  # Librería para visualización de datos
from sklearn.model_selection import train_test_split  # Función para dividir los datos en conjuntos de entrenamiento y prueba
from sklearn.feature_extraction.text import CountVectorizer  # Convertir una colección de documentos de texto en una matriz de tokens
from sklearn.naive_bayes import MultinomialNB  # Clasificador Naive Bayes
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métricas para evaluar el rendimiento del modelo
import nltk  # Librería para procesamiento de lenguaje natural
from nltk.corpus import stopwords  # Lista de palabras vacías en varios idiomas
from nltk.tokenize import word_tokenize  # Tokenización de palabras

# Descargar recursos necesarios de nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/izluis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/izluis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Leer Archivos de Datos
Leer los archivos cellphones_train.json y cellphones_test.json y crear los conjuntos de entrenamiento y prueba.

In [3]:
import json  # Librería para trabajar con archivos JSON

# Función para leer múltiples objetos JSON de un archivo
def read_json_objects(file_path):
    with open(file_path, 'r') as f:
        data = []
        for line in f:
            data.append(json.loads(line))
    return data

# Leer archivos JSON
train_data = read_json_objects('cellphones_train.json')
test_data = read_json_objects('cellphones_test.json')

# Crear DataFrames de pandas
df_train = pd.DataFrame(train_data)
df_test = pd.DataFrame(test_data)

# Mostrar las primeras filas de los DataFrames para verificar la carga correcta
df_train.head(), df_test.head()

(           categories                                id        klass  \
 0  [ "DEVICE", "OS" ]  d0fb202ea47c3cad448ea4e5e31bd404  information   
 1        [ "MOBILE" ]  d76f590d27f8eb415696e945f16b7e1c      neutral   
 2            [ "OS" ]  e345247011f2e69ca73ee5493c35e599     positive   
 3            [ "OS" ]  550f607acddf50d8dd515a750b994820  information   
 4        [ "MOBILE" ]  29e388d45111200b6535f1367c501461      neutral   
 
                                                 text     type  
 0  RT @matukpuntocom: Xperia Play se quedará sin ...   Xperia  
 1  RT @AlbertoCiurana: Hay gente que se desconect...  Celular  
 2  @loneliest_star siiiiiiiiiiiiiiii!!! y ya podr...  Android  
 3  El otro día soné que desarrollaba Apps para An...  Android  
 4  Escuchandola con Aristegui, lleva amistad BUEN...  Celular  ,
      categories                                id        klass  \
 0  [ "MOBILE" ]  0b4783023036948eed8f9f1151b708d8  information   
 1  [ "MOBILE" ]  fee4741dac44bf014

# Preprocesar Texto
Realizar preprocesamiento del texto, incluyendo la eliminación de stopwords, tokenización, y lematización.

In [4]:
import nltk  # Librería para procesamiento de lenguaje natural
from nltk.stem import WordNetLemmatizer  # Lematización de palabras

# Descargar recursos necesarios de nltk
nltk.download('wordnet')

# Inicializar lematizador
lemmatizer = WordNetLemmatizer()

# Función para preprocesar texto
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Tokenizar el texto
    tokens = word_tokenize(text)
    # Eliminar stopwords
    tokens = [word for word in tokens if word not in stopwords.words('spanish')]
    # Lematizar tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

# Aplicar preprocesamiento al conjunto de entrenamiento y prueba
df_train['processed_text'] = df_train['text'].apply(preprocess_text)
df_test['processed_text'] = df_test['text'].apply(preprocess_text)

# Mostrar las primeras filas de los DataFrames para verificar el preprocesamiento
df_train.head(), df_test.head()

[nltk_data] Downloading package wordnet to /Users/izluis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


KeyboardInterrupt: 

# Crear Gráficas de Palabras Comunes
Crear gráficas que muestren las palabras más comunes para cada una de las etiquetas usando matplotlib y seaborn.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Función para obtener las palabras más comunes
def obtener_palabras_comunes(textos, n=10):
    todas_las_palabras = ' '.join(textos).split()
    contador_palabras = Counter(todas_las_palabras)
    return contador_palabras.most_common(n)

# Obtener palabras más comunes para cada etiqueta
etiquetas = df_train['label'].unique()
palabras_comunes_por_etiqueta = {}

for etiqueta in etiquetas:
    textos_etiqueta = df_train[df_train['label'] == etiqueta]['processed_text']
    palabras_comunes_por_etiqueta[etiqueta] = obtener_palabras_comunes(textos_etiqueta)

# Crear gráficas de palabras comunes
fig, axes = plt.subplots(len(etiquetas), 1, figsize=(10, 5 * len(etiquetas)))

for i, etiqueta in enumerate(etiquetas):
    palabras, frecuencias = zip(*palabras_comunes_por_etiqueta[etiqueta])
    sns.barplot(x=frecuencias, y=palabras, ax=axes[i])
    axes[i].set_title(f'Palabras más comunes para la etiqueta: {etiqueta}')
    axes[i].set_xlabel('Frecuencia')
    axes[i].set_ylabel('Palabras')

plt.tight_layout()
plt.show()

# Vectorizar Texto
Vectorizar el texto usando CountVectorizer y TF-IDF.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  # Importar TF-IDF Vectorizer

# Inicializar CountVectorizer
count_vectorizer = CountVectorizer()
X_train_counts = count_vectorizer.fit_transform(df_train['processed_text'])
X_test_counts = count_vectorizer.transform(df_test['processed_text'])

# Inicializar TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(df_train['processed_text'])
X_test_tfidf = tfidf_vectorizer.transform(df_test['processed_text'])

# Mostrar las dimensiones de las matrices resultantes
X_train_counts.shape, X_test_counts.shape, X_train_tfidf.shape, X_test_tfidf.shape

# Entrenar Clasificador
Entrenar varios clasificadores como Regresión Logística, SVM, Naive Bayes, y Random Forest para predecir la etiqueta del texto.

In [ ]:
from sklearn.linear_model import LogisticRegression  # Importar Regresión Logística
from sklearn.svm import SVC  # Importar SVM
from sklearn.ensemble import RandomForestClassifier  # Importar Random Forest

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_val, y_train, y_val = train_test_split(X_train_tfidf, df_train['label'], test_size=0.2, random_state=42)

# Inicializar clasificadores
log_reg = LogisticRegression(max_iter=1000)
svm = SVC()
naive_bayes = MultinomialNB()
random_forest = RandomForestClassifier()

# Entrenar clasificadores
log_reg.fit(X_train, y_train)
svm.fit(X_train, y_train)
naive_bayes.fit(X_train, y_train)
random_forest.fit(X_train, y_train)

# Predecir etiquetas en el conjunto de validación
y_pred_log_reg = log_reg.predict(X_val)
y_pred_svm = svm.predict(X_val)
y_pred_naive_bayes = naive_bayes.predict(X_val)
y_pred_random_forest = random_forest.predict(X_val)

# Evaluar clasificadores
print("Logistic Regression")
print("Accuracy:", accuracy_score(y_val, y_pred_log_reg))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_log_reg))
print("Classification Report:\n", classification_report(y_val, y_pred_log_reg))

print("\nSupport Vector Machine")
print("Accuracy:", accuracy_score(y_val, y_pred_svm))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_svm))
print("Classification Report:\n", classification_report(y_val, y_pred_svm))

print("\nNaive Bayes")
print("Accuracy:", accuracy_score(y_val, y_pred_naive_bayes))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_naive_bayes))
print("Classification Report:\n", classification_report(y_val, y_pred_naive_bayes))

print("\nRandom Forest")
print("Accuracy:", accuracy_score(y_val, y_pred_random_forest))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_random_forest))
print("Classification Report:\n", classification_report(y_val, y_pred_random_forest))

# Calcular Accuracy
Calcular el accuracy de cada modelo en el conjunto de prueba.

In [ ]:
from sklearn.metrics import accuracy_score

# Predecir etiquetas en el conjunto de prueba
y_test_log_reg = log_reg.predict(X_test_tfidf)
y_test_svm = svm.predict(X_test_tfidf)
y_test_naive_bayes = naive_bayes.predict(X_test_tfidf)
y_test_random_forest = random_forest.predict(X_test_tfidf)

# Calcular accuracy para cada modelo
accuracy_log_reg = accuracy_score(df_test['label'], y_test_log_reg)
accuracy_svm = accuracy_score(df_test['label'], y_test_svm)
accuracy_naive_bayes = accuracy_score(df_test['label'], y_test_naive_bayes)
accuracy_random_forest = accuracy_score(df_test['label'], y_test_random_forest)

# Mostrar resultados
print("Accuracy Logistic Regression:", accuracy_log_reg)
print("Accuracy Support Vector Machine:", accuracy_svm)
print("Accuracy Naive Bayes:", accuracy_naive_bayes)
print("Accuracy Random Forest:", accuracy_random_forest)

# Documentación de Preprocesamiento y Modelos
Incluir un texto que documente los diferentes tipos de preprocesamiento usados, el tipo de vectorizador, el modelo para clasificar y su accuracy.

In [ ]:
# Documentación de Preprocesamiento y Modelos

"""
En esta sección, se documentan los diferentes tipos de preprocesamiento utilizados, el tipo de vectorizador, el modelo para clasificar y su accuracy.

## Preprocesamiento

1. **Conversión a minúsculas**: Todos los textos se convierten a minúsculas para asegurar la uniformidad.
2. **Tokenización**: Los textos se dividen en palabras individuales (tokens).
3. **Eliminación de stopwords**: Se eliminan las palabras vacías (stopwords) en español utilizando la lista de NLTK.
4. **Lematización**: Se aplica lematización a los tokens para reducirlos a su forma base.

## Vectorización

Se utilizaron dos tipos de vectorizadores para convertir los textos preprocesados en matrices de características:

1. **CountVectorizer**: Convierte una colección de documentos de texto en una matriz de tokens contados.
2. **TfidfVectorizer**: Convierte una colección de documentos de texto en una matriz de TF-IDF (Term Frequency-Inverse Document Frequency).

## Modelos de Clasificación

Se entrenaron y evaluaron cuatro modelos de clasificación diferentes:

1. **Regresión Logística (Logistic Regression)**
2. **Máquina de Soporte Vectorial (Support Vector Machine - SVM)**
3. **Naive Bayes Multinomial (Multinomial Naive Bayes)**
4. **Bosque Aleatorio (Random Forest)**

## Resultados de Accuracy

Los resultados de accuracy para cada modelo en el conjunto de prueba son los siguientes:

- **Logistic Regression**: {accuracy_log_reg}
- **Support Vector Machine**: {accuracy_svm}
- **Naive Bayes**: {accuracy_naive_bayes}
- **Random Forest**: {accuracy_random_forest}
"""

train_test_split